### 분할의 필요성
1. 핀포인트 정보 검색(정확성): 문서를 세분화함으로써 질문(Query) 에 연관성이 있는 정보만 가져오는데 도움이 됩니다. 각각의 단위는 특정 주제나 내용에 초점을 맞추므로, 관련성이 높은 정보를 제공합니다.

2. 리소스 최적화(효율성): 전체 문서를 LLM 으로 입력하게 되면 비용이 많이 발생할 뿐더러, 효율적인 답변을 많은 정보속에 발췌하여 답변하지 못하게 됩니다. 때로는 이러한 문제가 할루시네이션으로 이어지게 됩니다. 따라서, 답변에 필요한 정보만 발췌하기 위한 목적도 있습니다.

### 문서분할 과정
- 문서 구조 파악
  -> 어떤 단위로 나눌지 결정 (페이지별, 섹션별, 또는 문단별)  
  -> chunk size 몇 개의 토큰 단위로 나눌 것인지
  -> chunk overlap 맥락이 이어질 수 있도록 일부를 겹쳐서(overlap) 분할

In [ ]:
%pip install -qU langchain-text-splitters

In [ ]:
# 코랩의 기본 작업 폴더는 /content
# data/appendix-keywords.txt 파일을 열어서 f라는 파일 객체를 생성합니다.
with open("./data/appendix-keywords.txt") as f:
    file = f.read()  # 파일의 내용을 읽어서 file 변수에 저장합니다.

In [ ]:
# 파일으로부터 읽은 내용을 일부 출력합니다.
print(file[:500])

#물리적 기준 (단순 분할)

## 1. CharacterTextSplitter
단순히 지정한 글자 수 단위로 자릅니다. (가장 원시적)

텍스트를 청크(chunk)로 분할하는 코드->


In [ ]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter(
    # 텍스트를 분할할 때 사용할 구분자. 기본값은 "\n\n"입니다.
    # separator=" ",
    # 분할된 텍스트 청크의 최대 크기.
    chunk_size=250,
    # 분할된 텍스트 청크 간의 중복되는 문자 수
    chunk_overlap=50,
    # 텍스트의 길이를 계산하는 함수.
    length_function=len,
    # 구분자가 정규식인지 여부.
    is_separator_regex=False,
)

In [ ]:
# text_splitter를 사용하여 state_of_the_union 텍스트를 문서로 분할합니다.
texts = text_splitter.create_documents([file])
print(texts[0])  # 분할된 문서 중 첫 번째 문서를 출력합니다.

>첫번째 문서가 저렇게 된 이유는 **chunk_size를 맞추기 위해 다음 덩어리들도 붙여서**이다. CharacterTextSplitter는 구분자 기준으로 일단 나누고 설정한 chunk_size보다 작으면 size가 넘기 직전까지 합쳐버림.

In [ ]:
print(texts[1])

다음은 문서와 함께 메타데이터를 전달하는 예시입니다.

메타데이터가 문서와 함께 분할되는 점에 주목해 주세요.

create_documents 메서드는 텍스트 데이터와 메타데이터 리스트를 인자로 받습니다.

In [ ]:
metadatas = [
    {"document": 1},
    {"document": 2},
]  # 문서에 대한 메타데이터 리스트를 정의합니다.
documents = text_splitter.create_documents(
    [
        file,
        file,
    ],  # 분할할 텍스트 데이터를 리스트로 전달합니다.
    metadatas=metadatas,  # 각 문서에 해당하는 메타데이터를 전달합니다.
)
print(documents[0])  # 분할된 문서 중 첫 번째 문서를 출력합니다.


split_text() 메서드를 사용하여 텍스트를 분할합니다.

In [ ]:
# text_splitter를 사용하여 file 텍스트를 분할하고, 분할된 텍스트의 첫 번째 요소를 반환합니다.
text_splitter.split_text(file)[0]

Q. split_text랑 create_documents가 달라?

1. split_text (알맹이만!)
결과물: 문자열 리스트 ['텍스트1', '텍스트2', ...]

    특징: 원본 텍스트를 쪼갠 후, 순수한 **글자 데이터(String)**만 반환합니다.

    언제 써요? 단순히 쪼개진 글자들만 확인하고 싶을 때 사용합니다.

2. create_documents (포장지까지!)
결과물: 문서 객체 리스트 [Document(page_content='텍스트1', metadata={...}), ...]

    특징: 텍스트를 쪼갠 뒤, 이를 랭체인의 **Document 객체**로 변환해서 반환합니다.

    포장지(Metadata): 이때 파일 이름, 페이지 번호 같은 메타데이터를 함께 담을 수 있습니다.

    언제 써요? 나중에 **벡터 데이터베이스(Vector Store)**에 저장하거나, 출처를 남겨야 하는 복잡한 앱을 만들 때 무조건 사용합니다.

✂️ 텍스트 분할기에서 "정규식 여부"가 왜 중요할까?
1. 정규식이 아닐 때 (단순 문자)
구분자를 "\n\n"으로 설정하면, 정확히 엔터 두 번이 쳐진 곳만 찾아 자릅니다.

2. 정규식일 때 (패턴 지정) 🌟
훨씬 유연하게 자를 수 있습니다.

    예시: "\n+" (엔터가 한 번이든 백 번이든 상관없이 하나 이상 있으면 거기서 잘라줘!)

    예시: "(?<=\. )" (마침표와 공백이 있는 문장 단위를 찾아서 잘라줘!)

Q. 구분자가 정규식이 아닌데 그렇게 분할한 텍스트가 지정한 청크의 최대 크기보다 크다면 어떻게 돼? (과대 청크 대응 방식)

A. CharacterTextSplitter는 구분자를 찾지 못하면 자르지 못하고 chunk_size를 초과해 통째로 반환!

<다른 splitter>

02.	RecursiveCharacter 1순위 구분자가 없으면 공백, 그것도 없으면 글자 단위로 내려가며 어떻게든 chunk_size를 맞춥니다.
03.	TokenTextSplitter	가차 없음. 무조건 지정된 토큰 수만큼 칼같이 자릅니다.
04.	SemanticChunker	글자 수보다 문장 간의 '의미 유사도'가 우선입니다. 의미가 연결된다고 판단하면 chunk_size를 조금 넘기기도 합니다.
05.	코드 분할 (Code)	언어 문법 우선. 함수나 클래스 단위로 자르려 노력하지만, 한 함수가 너무 크면 내부 로직을 기준으로 강제 분할합니다.
06.	MarkdownHeader	헤더 기준. # 같은 헤더가 없으면 분할하지 않습니다. 보통 Recursive 분할기와 연결해서 2차 분할을 유도합니다.
07.	HTMLHeader	태그 기준. 06번과 비슷하게 태그가 없으면 덩어리가 커집니다. 단독 사용 시 chunk_size를 초과할 수 있습니다.
08.	RecursiveJson	구조 보존. JSON 키 값을 기준으로 쪼개되, 하나의 값이 너무 크면 그 값만 일반 텍스트 분할 방식으로 쪼갭니다.

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

# 1. 설정: 청크 크기는 10글자, 구분자는 텍스트에 없는 '★'
text_splitter = CharacterTextSplitter(
    separator="★",
    chunk_size=10,
    chunk_overlap=0,
    is_separator_regex=False # 정규식 사용 안 함
)

# 2. 샘플 텍스트 (총 20글자, '★' 없음)
sample_text = "이문장은총스무글자자리텍스트입니다."

# 3. 분할 실행
chunks = text_splitter.split_text(sample_text)

# 4. 결과 출력
for i, chunk in enumerate(chunks):
    print(f"청크 {i+1} ({len(chunk)}글자): {chunk}")

## 2. RecursiveCharacterTextSplitter

가장 많이 쓰임.

청크가 충분히 작아질 때까지 줄바꿈(\n), 마침표(.), 공백 순서(단락 -> 문장 -> 단어)대로 확인하며 단락이 깨지지 않게 최대한 노력하며 자름.

기본 문자 목록은 ["\n\n", "\n", " ", ""]

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    # 청크 크기를 매우 작게 설정합니다. 예시를 위한 설정입니다.
    chunk_size=250,
    chunk_overlap=50,
    length_function=len,
    is_separator_regex=False,
)

In [ ]:
# text_splitter를 사용하여 file 텍스트를 문서로 분할합니다.
texts = text_splitter.create_documents([file])
print(texts[0])  # 분할된 문서의 첫 번째 문서를 출력합니다.
print("===" * 20)
print(texts[1])  # 분할된 문서의 두 번째 문서를 출력합니다.

In [ ]:
# 텍스트를 분할하고 분할된 텍스트의 처음 2개 요소를 반환합니다.
text_splitter.split_text(file)[:2]

## 3. TokenTextSplitter
모델이 인식하는 '토큰' 개수 기준으로 자릅니다. 비용 계산이나 모델 입력 제한을 맞출 때 정확합니다.

### tiktoken
OpenAI에서 만든 빠른 BPE Tokenizer

`CharacterTextSplitter.from_tiktoken_encoder`: 문단/줄바꿈으로 자르되, 그 크기만 토큰으로 잽니다. (문맥 보호 + 토큰 제한 준수 둘 다 잡기 위함)

CharacterTextSplitter라는 껍데기를 쓰고, 알맹이(계산 방식)만 tiktoken으로 바꾼 하이브리드 방식

In [ ]:
from langchain_text_splitters import CharacterTextSplitter

text_splitter = CharacterTextSplitter.from_tiktoken_encoder(
    # 청크 크기를 300으로 설정합니다.
    chunk_size=300,
    # 청크 간 중복되는 부분이 없도록 설정합니다.
    chunk_overlap=0,
)
# file 텍스트를 청크 단위로 분할합니다.
texts = text_splitter.split_text(file)


문맥 보호를 위해 chunk_size가 넘었지만 그냥 끊지않고 그냥 저장하고 경고를 남김

In [ ]:
print(len(texts))  # 분할된 청크의 개수를 출력합니다.

### TokenTextSplitter

In [ ]:
from langchain_text_splitters import TokenTextSplitter

text_splitter = TokenTextSplitter(
    chunk_size=200,  # 청크 크기를 10으로 설정합니다.
    chunk_overlap=0,  # 청크 간 중복을 0으로 설정합니다.
)

# state_of_the_union 텍스트를 청크로 분할합니다.
texts = text_splitter.split_text(file)
print(texts[0])  # 분할된 텍스트의 첫 번째 청크를 출력합니다.

### spaCy
복잡한 언어 분석 작업을 엄청나게 빠른 속도로 수행

- 텍스트가 분할되는 방식: spaCy tokenizer에 의해 분할됩니다.
- chunk size가 측정되는 방법: **문자 수**로 측정됩니다.

> 이게 좋은 이유!
1. 언어의 이해: 문법적으로 문장이 끝나는 지점을 정확히 찾음
    
    예: "U.S.A.에 갔다. 거기서 친구를 만났다." 👉 일반 분할기는 U.S.A. 뒤의 마침표에서 문장을 잘라버릴 수 있지만, spaCy는 이게 약어임을 알고 문장 끝까지 유지합니다.

2. 맥락 보존: LLM이 이해하기에 훨씬 좋음.

en_core_web_sm 모델을 다운로드

In [ ]:
!python -m spacy download en_core_web_sm --quiet

In [ ]:
import spacy

# 이 명령어로 직접 로드하면 런타임 재시작 없이 바로 사용할 수 있습니다.
nlp = spacy.load("en_core_web_sm")

In [ ]:
import warnings
from langchain_text_splitters import SpacyTextSplitter

# 경고 메시지를 무시합니다.
warnings.filterwarnings("ignore")

# SpacyTextSplitter를 생성합니다.
text_splitter = SpacyTextSplitter(
    chunk_size=200,  # 청크 크기를 200으로 설정합니다.
    chunk_overlap=50,  # 청크 간 중복을 50으로 설정합니다.
)

In [ ]:
# text_splitter를 사용하여 file 텍스트를 분할합니다.
texts = text_splitter.split_text(file)
print(texts[0])  # 분할된 텍스트의 첫 번째 요소를 출력합니다.


### SentenceTransformers
텍스트의 '의미'를 숫자(vector)로 바꿔서 AI가 맥락을 파악하게 돕는 핵심 엔진

In [ ]:
from langchain_text_splitters import SentenceTransformersTokenTextSplitter

# 문장 분할기를 생성하고 청크 간 중복을 0으로 설정합니다.
splitter = SentenceTransformersTokenTextSplitter(chunk_size=200, chunk_overlap=0)

특수 토큰 제외
* [CLS]: 문장의 시작 (Start)
* [SEP]: 문장의 끝 (Stop)

In [ ]:
count_start_and_stop_tokens = 2  # 시작과 종료 토큰의 개수를 2로 설정합니다.

# 텍스트의 토큰 개수에서 시작과 종료 토큰의 개수를 뺍니다.
text_token_count = splitter.count_tokens(
    text=file) - count_start_and_stop_tokens
print(text_token_count)  # 계산된 텍스트 토큰 개수를 출력합니다.

In [ ]:
text_chunks = splitter.split_text(text=file)  # 텍스트를 청크로 분할합니다.
print(text_chunks[1])  # 분할된 텍스트 청크 중 두 번째 청크를 출력합니다.

### NLTK (Natural Language Toolkit)
파이썬에서 가장 오래되고 유명한, 영어 자연어 처리(NLP)를 위한 라이브러리와 프로그램 모음

spaCy가 '실무와 속도'에 집중했다면(가장 좋은 로직 하나를 제공함), NLTK는 **'교육용, 연구용 (다양한 기능)'**에 강점(사용자가 로직을 직접 선택함)!

사람이 언어를 분석하는 거의 모든 방법(단어 나누기, 문법 분석, 감정 분석 등)이 포함되어 있고 언어 모델을 학습시키거나 테스트할 때 필요한 다양한 말뭉치(Corpus)를 기본적으로 제공함.

### KoNLPy(Korean NLP in Python)
한국어 자연어 처리(NLP)를 위한 파이썬 패키지

토큰 분할의 효과는 언어 구조에 대한 토크나이저의 이해에 크게 의존하며, 이는 의미 있는 토큰 생성을 보장합니다.

영어를 위해 설계된 토크나이저는 한국어와 같은 다른 언어의 고유한 의미 구조를 이해할 수 있는 능력이 없기 때문에 한국어 처리에 효과적으로 사용될 수 없습니다.

 KoNLPY에는 Kkma(Korean Knowledge Morpheme Analyzer)라는 형태소 분석기가 포함.

 Kkma는 한국어 텍스트에 대한 상세한 형태소 분석을 제공하는데 이 정밀함이 처리 속도에 영향을 미칠 수 있음을 유의.

### Hugging Face tokenizer
Hugging Face는 다양한 토크나이저를 제공

# 의미론적 유사성 기준

## 4. 시멘틱 청커(SemanticChunker)
글자 수가 아니라 **"내용의 의미가 변하는 지점"**을 찾아서 자릅니다. AI(임베딩)를 사용해 문장 간의 유사도를 측정하므로, 문맥이 끊기지 않아 답변의 품질이 가장 높습니다.

In [ ]:
%pip install -qU langchain_experimental langchain_openai

In [ ]:
from google.colab import userdata
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

`SemanticChunker`는 LangChain의 실험적 기능 중 하나로, 텍스트를 의미론적으로 유사한 청크로 분할하는 역할을 합니다.

#### Breakpoints
문장을 "분리"할 시점

##### 1. Percentile

기본적인 분리 방식은 백분위수(Percentile) 를 기반으로 합니다.

이 방법에서는 문장 간의 모든 차이를 계산한 다음, 지정한 백분위수를 기준으로 분리합니다.

In [ ]:
from langchain_experimental.text_splitter import SemanticChunker
from langchain_openai.embeddings import OpenAIEmbeddings

# OpenAI 임베딩을 사용하여 의미론적 청크 분할기를 초기화합니다.
text_splitter = SemanticChunker(
    # OpenAI의 임베딩 모델을 사용하여 시맨틱 청커를 초기화합니다.
    OpenAIEmbeddings(),
    # 분할 기준점 유형을 백분위수로 설정합니다.
    breakpoint_threshold_type="percentile",
    breakpoint_threshold_amount=70,
)

In [ ]:
docs = text_splitter.create_documents([file])
for i, doc in enumerate(docs[:5]):
    print(f"[Chunk {i}]", end="\n\n")
    print(doc.page_content)  # 분할된 문서 중 첫 번째 문서의 내용을 출력합니다.
    print("===" * 20)

엉망;;;

##### 2. Standard Deviation

In [ ]:
text_splitter = SemanticChunker(
    # OpenAI의 임베딩 모델을 사용하여 시맨틱 청커를 초기화합니다.
    OpenAIEmbeddings(),
    # 분할 기준으로 표준 편차를 사용합니다.
    breakpoint_threshold_type="standard_deviation",
    breakpoint_threshold_amount=1.25,
)
docs = text_splitter.create_documents([file])

In [ ]:
docs = text_splitter.create_documents([file])
for i, doc in enumerate(docs[:5]):
    print(f"[Chunk {i}]", end="\n\n")
    print(doc.page_content)  # 분할된 문서 중 첫 번째 문서의 내용을 출력합니다.
    print("===" * 20)

##### 3. Interquartile

In [ ]:
text_splitter = SemanticChunker(
    # OpenAI의 임베딩 모델을 사용하여 의미론적 청크 분할기를 초기화합니다.
    OpenAIEmbeddings(),
    # 분할 기준점 임계값 유형을 사분위수 범위로 설정합니다.
    breakpoint_threshold_type="interquartile",
    breakpoint_threshold_amount=0.5,
)
# text_splitter를 사용하여 분할합니다.
docs = text_splitter.create_documents([file])

# 결과를 출력합니다.
for i, doc in enumerate(docs[:5]):
    print(f"[Chunk {i}]", end="\n\n")
    print(doc.page_content)  # 분할된 문서 중 첫 번째 문서의 내용을 출력합니다.
    print("===" * 20)

# 구조적 기준 (문서 형식 활용)

## 5. Code 분할
Python, JAVA 등 언어별 문법을 인식하여 함수나 클래스 단위로 자릅니다.

#### Split code

In [ ]:
from langchain_text_splitters import (
    Language,
    RecursiveCharacterTextSplitter,
)
# 지원되는 언어의 전체 목록을 가져옵니다.
[e.value for e in Language]
# 주어진 언어에 대해 사용되는 구분자를 확인할 수 있습니다.
RecursiveCharacterTextSplitter.get_separators_for_language(Language.PYTHON)


#### Python

In [ ]:
PYTHON_CODE = """
def hello_world():
    print("Hello, World!")

hello_world()
"""

python_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=50, chunk_overlap=0
)
python_docs = python_splitter.create_documents([PYTHON_CODE])
python_docs


In [ ]:
for doc in python_docs:
    print(doc.page_content, end="\n==================\n")


## 6. Markdown 헤더 분할
->#, ## 같은 제목 수준에 맞춰 내용을 나눕니다.

MarkdownHeaderTextSplitter를 사용하여 마크다운 형식의 텍스트를 헤더 단위로 분할합니다.

마크다운 문서의 헤더(#, ##, ### 등)를 기준으로 텍스트를 분할하는 역할을 합니다.

markdown_document 변수에 마크다운 형식의 문서가 할당됩니다.

headers_to_split_on 리스트에는 마크다운 헤더 레벨과 해당 레벨의 이름이 튜플 형태로 정의됩니다.
MarkdownHeaderTextSplitter 클래스를 사용하여 markdown_splitter 객체를 생성하며, headers_to_split_on 매개변수로 분할 기준이 되는 헤더 레벨을 전달합니다.
split_text 메서드를 호출하여 markdown_document를 헤더 레벨에 따라 분할합니다.

In [ ]:
from langchain_text_splitters import MarkdownHeaderTextSplitter

# 마크다운 형식의 문서를 문자열로 정의합니다.
markdown_document = "# Title\n\n## 1. SubTitle\n\nHi this is Jim\n\nHi this is Joe\n\n### 1-1. Sub-SubTitle \n\nHi this is Lance \n\n## 2. Baz\n\nHi this is Molly"
print(markdown_document)


In [ ]:
headers_to_split_on = [  # 문서를 분할할 헤더 레벨과 해당 레벨의 이름을 정의합니다.
    (
        "#",
        "Header 1",
    ),  # 헤더 레벨 1은 '#'로 표시되며, 'Header 1'이라는 이름을 가집니다.
    (
        "##",
        "Header 2",
    ),  # 헤더 레벨 2는 '##'로 표시되며, 'Header 2'라는 이름을 가집니다.
    (
        "###",
        "Header 3",
    ),  # 헤더 레벨 3은 '###'로 표시되며, 'Header 3'이라는 이름을 가집니다.
]

# 마크다운 헤더를 기준으로 텍스트를 분할하는 MarkdownHeaderTextSplitter 객체를 생성합니다.
markdown_splitter = MarkdownHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
# markdown_document를 헤더를 기준으로 분할하여 md_header_splits에 저장합니다.
md_header_splits = markdown_splitter.split_text(markdown_document)
# 분할된 결과를 출력합니다.
for header in md_header_splits:
    print(f"{header.page_content}")
    print(f"{header.metadata}", end="\n=====================\n")


In [ ]:
markdown_splitter = MarkdownHeaderTextSplitter(
    # 분할할 헤더를 지정합니다.
    headers_to_split_on=headers_to_split_on,
    # 헤더를 제거하지 않도록 설정합니다.
    strip_headers=False,
)
# 마크다운 문서를 헤더를 기준으로 분할합니다.
md_header_splits = markdown_splitter.split_text(markdown_document)
# 분할된 결과를 출력합니다.
for header in md_header_splits:
    print(f"{header.page_content}")
    print(f"{header.metadata}", end="\n=====================\n")


In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

markdown_document = "# Intro \n\n## History \n\nMarkdown[9] is a lightweight markup language for creating formatted text using a plain-text editor. John Gruber created Markdown in 2004 as a markup language that is appealing to human readers in its source code form.[9] \n\nMarkdown is widely used in blogging, instant messaging, online forums, collaborative software, documentation pages, and readme files. \n\n## Rise and divergence \n\nAs Markdown popularity grew rapidly, many Markdown implementations appeared, driven mostly by the need for \n\nadditional features such as tables, footnotes, definition lists,[note 1] and Markdown inside HTML blocks. \n\n#### Standardization \n\nFrom 2012, a group of people, including Jeff Atwood and John MacFarlane, launched what Atwood characterised as a standardisation effort. \n\n## Implementations \n\nImplementations of Markdown are available for over a dozen programming languages."
print(markdown_document)


## 7. HTML 헤더 분할
\<h1>, \<h2> 태그를 기준으로 웹 페이지 내용을 나눕니다.

MarkdownHeaderTextSplitter와 개념적으로 유사한 HTMLHeaderTextSplitter는 텍스트를 요소 수준에서 분할하고 각 헤더에 대한 메타데이터를 추가하는 "구조 인식" 청크 생성기입니다.

이는 각 청크와 "관련된" 메타데이터를 추가합니다.

HTMLHeaderTextSplitter는 요소별로 청크를 반환하거나 동일한 메타데이터를 가진 요소를 결합할 수 있으며

(a) 관련 텍스트를 의미론적으로 (대략적으로) 그룹화하고
(b) 문서 구조에 인코딩된 컨텍스트 풍부한 정보를 보존하는 것을 목표로 합니다.

In [ ]:
from langchain_text_splitters import HTMLHeaderTextSplitter

html_string = """
<!DOCTYPE html>
<html>
<body>
    <div>
        <h1>Foo</h1>
        <p>Some intro text about Foo.</p>
        <div>
            <h2>Bar main section</h2>
            <p>Some intro text about Bar.</p>
            <h3>Bar subsection 1</h3>
            <p>Some text about the first subtopic of Bar.</p>
            <h3>Bar subsection 2</h3>
            <p>Some text about the second subtopic of Bar.</p>
        </div>
        <div>
            <h2>Baz</h2>
            <p>Some text about Baz</p>
        </div>
        <br>
        <p>Some concluding text about Foo</p>
    </div>
</body>
</html>
"""

headers_to_split_on = [
    ("h1", "Header 1"),  # 분할할 헤더 태그와 해당 헤더의 이름을 지정합니다.
    ("h2", "Header 2"),
    ("h3", "Header 3"),
]

# 지정된 헤더를 기준으로 HTML 텍스트를 분할하는 HTMLHeaderTextSplitter 객체를 생성합니다.
html_splitter = HTMLHeaderTextSplitter(headers_to_split_on=headers_to_split_on)
# HTML 문자열을 분할하여 결과를 html_header_splits 변수에 저장합니다.
html_header_splits = html_splitter.split_text(html_string)
# 분할된 결과를 출력합니다.
for header in html_header_splits:
    print(f"{header.page_content}")
    print(f"{header.metadata}", end="\n=====================\n")


## 8. 재귀적 JSON 분할
복잡한 JSON 데이터를 구조를 유지하며 작은 덩어리로 쪼갭니다.

구조를 유지하는게 왜 중요하냐면,
JSON은 { }와 [ ]가 겹겹이 쌓인 구조라서 글자 수로 단순히 나눠버리면 {는 있는데  }가 없어서 망가진 JSON이 되거나 문맥이 상실될 수 있음;;

